# Data Assembly 

In [8]:
import os
import pandas as pd
import numpy as np

# Set working directory 

os.chdir("C:\\Users\\yonas\\Documents\\ICPAC\\ea_seasonal_pred\\seasonal-ml-pred")


PROCESSED_DIR = "data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)


In [14]:
# Load processed dataframes
OND_uganda = pd.read_csv("data//processed//uganda_OND_1981_2024.csv", index_col=0)
MAM_uganda = pd.read_csv("data//processed//uganda_MAM_1981_2024.csv", index_col=0)

In [15]:
OND_uganda.shape, MAM_uganda.shape

((44, 4), (44, 4))

In [16]:
def missing_report(df: pd.DataFrame, title: str = ""):
    col_na = df.isna().sum().sort_values(ascending=False)
    col_rate = (df.isna().mean()*100).round(1).sort_values(ascending=False)
    row_na = df.isna().sum(axis=1)
    row_rate = (df.isna().mean(axis=1)*100).round(1)
    print(f"=== Missingness report: {title} ===")
    print(f"Rows: {len(df)} | Cols: {df.shape[1]}")
    print(f"Rows with ≥1 missing: {int((row_na>0).sum())} ({(row_na>0).mean()*100:.1f}%)")
    print("\nTop 10 columns by % missing:")
    display(col_rate.head(10).to_frame("% missing"))
    print("\nYears with any missing (first 10):")
    display(row_rate[row_rate>0].head(10).to_frame("% missing in row"))

In [17]:
missing_report(OND_uganda, "OND 1981-2024")

=== Missingness report: OND 1981-2024 ===
Rows: 44 | Cols: 4
Rows with ≥1 missing: 0 (0.0%)

Top 10 columns by % missing:


,% missing
OND_total_mm,0.0
OND_clim1991_2020_mm,0.0
OND_anom_mm,0.0
OND_anom_std,0.0



Years with any missing (first 10):


,% missing in row
season_year,


In [18]:
missing_report(MAM_uganda, "MAM 1981-2024")

=== Missingness report: MAM 1981-2024 ===
Rows: 44 | Cols: 4
Rows with ≥1 missing: 0 (0.0%)

Top 10 columns by % missing:


,% missing
MAM_total_mm,0.0
MAM_clim1991_2020_mm,0.0
MAM_anom_mm,0.0
MAM_anom_std,0.0



Years with any missing (first 10):


,% missing in row
season_year,


In [9]:
# Load the features dataframes
features_OND_all_plus = pd.read_csv("data//processed//features_OND_all_plus_persist.csv", index_col=0)
features_MAM_all_plus = pd.read_csv("data//processed//features_MAM_all_plus_persist.csv", index_col=0)

In [13]:
features_OND_all_plus.shape, features_MAM_all_plus.shape

((44, 30), (44, 30))

In [ ]:
missing_report(, "Features OND all plus")features_OND_all_plus

=== Missingness report: Features OND all plus ===
Rows: 44 | Cols: 30
Rows with ≥1 missing: 0 (0.0%)

Top 10 columns by % missing:


,% missing
n12_JAS,0.0
n12_AS,0.0
n12_Sep,0.0
n3_JAS,0.0
n3_AS,0.0
n3_Sep,0.0
n34_JAS,0.0
n34_AS,0.0
n34_Sep,0.0
n4_JAS,0.0



Years with any missing (first 10):


,% missing in row
season_year,


In [20]:
missing_report(features_MAM_all_plus, "MAM features all plus")

=== Missingness report: MAM features all plus ===
Rows: 44 | Cols: 30
Rows with ≥1 missing: 1 (2.3%)

Top 10 columns by % missing:


,% missing
n12_NDJ,2.3
n12_DJ,2.3
n3_NDJ,2.3
n3_DJ,2.3
n34_DJ,2.3
n34_NDJ,2.3
meiv2_DJ,2.3
meiv2_NDJ,2.3
n4_NDJ,2.3
n4_DJ,2.3



Years with any missing (first 10):


,% missing in row
season_year,
1981,70.0


In [21]:
# Lets drop the the 1981 rows since they have alot of missing data
features_MAM_all_plus = features_MAM_all_plus[features_MAM_all_plus.index >= 1982]
features_MAM_all_plus.head()

,n12_NDJ,n12_DJ,n12_Feb,n3_NDJ,n3_DJ,n3_Feb,n34_NDJ,n34_DJ,n34_Feb,n4_NDJ,...,pacwarmpool_Feb,censo_NDJ,censo_DJ,censo_Feb,dmi_NDJ,dmi_DJ,dmi_Feb,PERSIST_SONprev_anom_mm,PERSIST_ONDprev_anom_mm,PERSIST_DJF_anom_mm
season_year,,,,,,,,,,,,,,,,,,,,,
1982,-0.300000,-0.220,-0.79,-0.086667,0.075,-0.00,-0.160000,-0.035,-0.17,-0.143333,...,-0.305,-0.166667,-0.175,-0.04,-0.067667,0.0625,0.166,1.067961,-3.141819,-1.641512
1983,2.946667,2.905,2.03,2.753333,2.930,2.30,2.156667,2.275,1.94,0.490000,...,-0.153,2.503333,2.440,2.76,-0.120000,-0.3220,-0.587,-0.035007,5.855969,-0.308066
1984,0.073333,-0.090,-0.88,-0.816667,-0.710,-0.15,-0.996667,-0.850,-0.19,-0.890000,...,-0.404,-0.390000,-0.330,-0.47,-0.222333,-0.1620,-0.149,-4.057695,-3.513931,0.486139
1985,-0.436667,-0.590,-1.38,-1.173333,-1.265,-0.90,-1.230000,-1.250,-0.72,-0.833333,...,-0.275,-0.463333,-0.375,-0.84,-0.387333,-0.3720,-0.627,-5.366506,-2.751584,1.288048
1986,-0.390000,-0.190,-0.10,-0.743333,-0.765,-0.53,-0.480000,-0.535,-0.71,-0.280000,...,-0.318,-0.280000,-0.410,0.17,-0.152000,-0.2650,-0.135,-3.723882,-7.442914,-2.746658


In [22]:
missing_report(features_MAM_all_plus, "MAM features all plus")

=== Missingness report: MAM features all plus ===
Rows: 43 | Cols: 30
Rows with ≥1 missing: 0 (0.0%)

Top 10 columns by % missing:


,% missing
n12_NDJ,0.0
n12_DJ,0.0
n12_Feb,0.0
n3_NDJ,0.0
n3_DJ,0.0
n3_Feb,0.0
n34_NDJ,0.0
n34_DJ,0.0
n34_Feb,0.0
n4_NDJ,0.0



Years with any missing (first 10):


,% missing in row
season_year,


In [23]:
# Drop the 1981 rows from the MAM_uganda dataframe as well
MAM_uganda = MAM_uganda[MAM_uganda.index >= 1982]
MAM_uganda.head()

,MAM_total_mm,MAM_clim1991_2020_mm,MAM_anom_mm,MAM_anom_std
season_year,,,,
1982,435.96973,416.46323,19.506500,0.380850
1983,354.69670,416.46323,-61.766540,-1.205945
1984,312.64685,416.46323,-103.816376,-2.026936
1985,479.86670,416.46323,63.403473,1.237905
1986,415.87973,416.46323,-0.583496,-0.011392


In [ ]:
def load_target(name_parq: str, name_csv: str, cols_keep: list[str]) -> pd.DataFrame:
    parq_path = os.path.join(PROCESSED_DIR, name_parq)
    csv_path  = os.path.join(PROCESSED_DIR, name_csv)
    if os.path.exists(parq_path):
        df = pd.read_parquet(parq_path)
    elif os.path.exists(csv_path):
        df = pd.read_csv(csv_path, index_col=0)
    else:
        raise FileNotFoundError(f"Missing target file: {parq_path} or {csv_path}")
    # ensure index is season_year (int)
    if df.index.name != "season_year":
        df.index.name = "season_year"
    try:
        df.index = df.index.astype(int)
    except Exception:
        pass
    # keep only requested columns if present
    cols = [c for c in cols_keep if c in df.columns]
    return df[cols].sort_index()

In [25]:


OND_cols = ["OND_total_mm", "OND_clim1991_2020_mm", "OND_anom_mm", "OND_anom_std"]
MAM_cols = ["MAM_total_mm", "MAM_clim1991_2020_mm", "MAM_anom_mm", "MAM_anom_std"]

OND_tgt = load_target("kenya_OND_1981_2024.parquet", "uganda_OND_1981_2024.csv", OND_cols)
MAM_tgt = load_target("kenya_MAM_1981_2024.parquet", "uganda_MAM_1981_2024.csv", MAM_cols)

print("OND target years:", OND_tgt.index.min(), "→", OND_tgt.index.max(), "| rows:", len(OND_tgt))
print("MAM target years:", MAM_tgt.index.min(), "→", MAM_tgt.index.max(), "| rows:", len(MAM_tgt))
OND_tgt.head(2), MAM_tgt.head(2)


OND target years: 1981 → 2024 | rows: 44
MAM target years: 1981 → 2024 | rows: 44


(             OND_total_mm  OND_clim1991_2020_mm  OND_anom_mm  OND_anom_std
 season_year                                                               
 1981            251.31606              328.4043    -77.08824     -1.053309
 1982            373.33360              328.4043     44.92929      0.613899,
              MAM_total_mm  MAM_clim1991_2020_mm  MAM_anom_mm  MAM_anom_std
 season_year                                                               
 1981            459.40836             416.46323     42.94513      0.838471
 1982            435.96973             416.46323     19.50650      0.380850)

In [26]:
def load_features(primary_parq: str, fallback_parq: str) -> pd.DataFrame:
    primary = os.path.join(PROCESSED_DIR, primary_parq)
    fallback = os.path.join(PROCESSED_DIR, fallback_parq)
    if os.path.exists(primary):
        df = pd.read_parquet(primary)
    elif os.path.exists(fallback):
        df = pd.read_parquet(fallback)
    else:
        raise FileNotFoundError(f"Missing features: {primary} or {fallback}")
    # ensure index is season_year
    if df.index.name != "season_year":
        df.index.name = "season_year"
    try:
        df.index = df.index.astype(int)
    except Exception:
        pass
    return df.sort_index()

In [30]:
OND_feat = features_OND_all_plus
MAM_feat = features_MAM_all_plus

In [31]:
# Overlap years
years_ond = OND_tgt.index.intersection(OND_feat.index)
years_mam = MAM_tgt.index.intersection(MAM_feat.index)

In [32]:
years_ond

Index([1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype='int64', name='season_year')

In [43]:
OND_tgt.index.min(), OND_tgt.index.max()

(np.int64(1981), np.int64(2024))

In [33]:
years_mam

Index([1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype='int64', name='season_year')

In [44]:
MAM_tgt.index.min(), MAM_tgt.index.max()

(np.int64(1981), np.int64(2024))

In [ ]:
# Inner join (keeps years present in both)
OND_table = OND_tgt.loc[years_ond].join(OND_feat.loc[years_ond], how="inner")
MAM_table = MAM_tgt.loc[years_mam].join(MAM_feat.loc[years_mam], how="inner")


In [38]:
OND_table.shape, MAM_table.shape

((44, 34), (43, 34))

In [45]:
OND_table.index.min(), OND_table.index.max()

(np.int64(1981), np.int64(2024))

In [36]:
OND_table.head(2)

,OND_total_mm,OND_clim1991_2020_mm,OND_anom_mm,OND_anom_std,n12_JAS,n12_AS,n12_Sep,n3_JAS,n3_AS,n3_Sep,...,pacwarmpool_Sep,censo_JAS,censo_AS,censo_Sep,dmi_JAS,dmi_AS,dmi_Sep,PERSIST_MJJ_anom_mm,PERSIST_JJA_anom_mm,PERSIST_JAS_anom_mm
season_year,,,,,,,,,,,,,,,,,,,,,
1981,251.31606,328.4043,-77.08824,-1.053309,-0.936667,-0.960,-0.79,-0.520000,-0.48,-0.23,...,-0.186,-0.360000,-0.235,-0.18,-0.648333,-0.6925,-0.757,-2.042121,1.513391,3.086952
1982,373.33360,328.4043,44.92929,0.613899,0.860000,1.055,1.31,1.083333,1.42,1.89,...,-0.495,1.786667,1.905,1.95,0.321000,0.3490,0.442,-8.787225,-1.924204,-5.021202


In [37]:
MAM_table.head(2)


,MAM_total_mm,MAM_clim1991_2020_mm,MAM_anom_mm,MAM_anom_std,n12_NDJ,n12_DJ,n12_Feb,n3_NDJ,n3_DJ,n3_Feb,...,pacwarmpool_Feb,censo_NDJ,censo_DJ,censo_Feb,dmi_NDJ,dmi_DJ,dmi_Feb,PERSIST_SONprev_anom_mm,PERSIST_ONDprev_anom_mm,PERSIST_DJF_anom_mm
season_year,,,,,,,,,,,,,,,,,,,,,
1982,435.96973,416.46323,19.50650,0.380850,-0.300000,-0.220,-0.79,-0.086667,0.075,-0.0,...,-0.305,-0.166667,-0.175,-0.04,-0.067667,0.0625,0.166,1.067961,-3.141819,-1.641512
1983,354.69670,416.46323,-61.76654,-1.205945,2.946667,2.905,2.03,2.753333,2.930,2.3,...,-0.153,2.503333,2.440,2.76,-0.120000,-0.3220,-0.587,-0.035007,5.855969,-0.308066


In [46]:
MAM_table.index.min(), MAM_table.index.max()

(np.int64(1982), np.int64(2024))

In [ ]:
# Export final data tables as CSV files

OND_table.to_csv(os.path.join(PROCESSED_DIR, "uganda_OND_target_features_1981_2024.csv"), index=True)
MAM_table.to_csv(os.path.join(PROCESSED_DIR, "uganda_MAM_target_features_1982_2024.csv"), index=True)
